0. 데이터 설명
- CSI300은 중국 KOSPI 같은 종목으로 대표 종목 300개로 구성되어 있음
- 중국 데이터는 2016년도부터 2023년 4월까지 있음
- CIS300股票数据2016.5-2023.4.csv 파일에 전체 데이터 수록
- 총데이터를 종목 코드값을 기준

1.chinaStockRestructure 
- 전체 주식 데이터를 종목별로 재구성하는 코드
- CIS300股票数据2016.5-2023.4.csv에 있는 파일을 '종목코드.CSV'로 변경
 
2. inputMaker
- chinaStockRestructure로 '종목코드.CSV' 구성된 파일이 있는 폴더 읽음
- 해당 폴더에 있는 전 종목을 읽으며 입력피쳐 생성함
- 종목별  dataframe 생성하고 입력피쳐 계산 (해당 코드 커스텀)
- 입력피쳐가 추가된 전종목 데이터들을 AllStock 이라는 데이터프레임에 넣고 7:2:1로 분할 (학습/검증/테스트 - 비율 조정 가능)
- 정리된 학습/검증/테스트 파일을 저장하는 코드도 있는데 현재 비활성화
- with tf.device('/GPU:0'): model = Sequential() ~ 코드부터 모델 구조 (해당 코드 커스텀)

In [1]:
import os
import pandas as pd
import numpy as np

# 파일이 있는 디렉토리 경로 설정
directory_path = r'D:\주식데이터\CSI300_NAME'
AllStock = pd.DataFrame() #피쳐 만들어진 종목들 취합용
engine_name = "china_stock"
# 디렉토리 내의 모든 CSV 파일을 읽어서 처리
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)

        # CSV 파일을 데이터프레임으로 읽기
        df = pd.read_csv(file_path, encoding='utf-8-sig')  # 필요한 인코딩을 사용하세요
        
       
        # 여기에 원하는 피쳐 계산식 추가 
        # 날짜를 날짜 형식으로 변경
        df['TradingDate'] = pd.to_datetime(df['TradingDate'])
        
        ##--- 가격/거래량 이동 평균 관련 피쳐 ---##
        moving_average_period = [5,10,20,60,120] #이평선 구하고싶은 수치 변경 가능
        for period in moving_average_period:        
            #이동평균선 계산
            df['MA'+str(period)] = df.groupby('Symbol')['ClosePrice'].transform(lambda x: x.rolling(period).mean())
            #이동평균선의 기울기 계산
            df['Grad'+str(period)] = df.groupby('Symbol')['MA'+str(period)].transform(lambda x: np.gradient(x))
            #거래량 이동평균선 계산
            df['VMA'+str(period)] = df.groupby('Symbol')['Volume'].transform(lambda x: x.rolling(period).mean())
            #거래량 이동평균선의 기울기 계산
            df['VGrad'+str(period)] = df.groupby('Symbol')['VMA'+str(period)].transform(lambda x: np.gradient(x))
            
            ##--- 이격도 관련 피쳐 ---##
            #이동평균선과 현재 가격의 차이 계산           
            df['Dis'+str(period)] = (df['ClosePrice'] / df['MA'+str(period)]) * 100
            df['Div'+str(period)] = df['ClosePrice'] - df['MA'+str(period)] 
            df['VDis'+str(period)] = (df['Volume'] / df['VMA'+str(period)]) * 100
            df['VDiv'+str(period)] = df['Volume'] - df['VMA'+str(period)] 
            
        ##--- 일별 음봉 양봉 정보 ---# (1이면 양봉, 0이면 음봉 의미)
        df['CandleType'] = df.apply(lambda row: '1' if row['ClosePrice'] > row['OpenPrice'] else '0', axis=1) 
        

        ##--- 가격 변화량  ---# 
        df['PriceChange'] = df.groupby('Symbol')['ClosePrice'].pct_change(periods=5)*100  # 이전 5일 동안의 상승률 계산
        
        
        # 상승률이 10% 이상인 행 선택 : 이전 5일 동안 종가 상승률이 10% 이상인 날짜 1, 아니면 0 
        df['Target'] = df['PriceChange'].apply(lambda x: 1 if x >= 1.0 else 0)
        
        #몇 가지 더 개발해서 추가! 


        df = df.fillna(0) #NaN 데이터 모두 0으로 초기화
        
        #종목별로 나눠졌던 데이터를 합쳐?
        AllStock = pd.concat([AllStock, df], axis=0)
        print('----'+os.path.splitext(filename)[0] +' 종목 입력피쳐 계산 완료 ----')
      
        
        

----000001 종목 입력피쳐 계산 완료 ----
----000002 종목 입력피쳐 계산 완료 ----
----000063 종목 입력피쳐 계산 완료 ----
----000069 종목 입력피쳐 계산 완료 ----
----000100 종목 입력피쳐 계산 완료 ----
----000157 종목 입력피쳐 계산 완료 ----
----000166 종목 입력피쳐 계산 완료 ----
----000301 종목 입력피쳐 계산 완료 ----
----000333 종목 입력피쳐 계산 완료 ----
----000338 종목 입력피쳐 계산 완료 ----
----000408 종목 입력피쳐 계산 완료 ----
----000425 종목 입력피쳐 계산 완료 ----
----000538 종목 입력피쳐 계산 완료 ----
----000568 종목 입력피쳐 계산 완료 ----
----000596 종목 입력피쳐 계산 완료 ----
----000625 종목 입력피쳐 계산 완료 ----
----000651 종목 입력피쳐 계산 완료 ----
----000661 종목 입력피쳐 계산 완료 ----
----000708 종목 입력피쳐 계산 완료 ----
----000723 종목 입력피쳐 계산 완료 ----
----000725 종목 입력피쳐 계산 완료 ----
----000733 종목 입력피쳐 계산 완료 ----
----000768 종목 입력피쳐 계산 완료 ----
----000776 종목 입력피쳐 계산 완료 ----
----000786 종목 입력피쳐 계산 완료 ----
----000792 종목 입력피쳐 계산 완료 ----
----000800 종목 입력피쳐 계산 완료 ----
----000858 종목 입력피쳐 계산 완료 ----
----000876 종목 입력피쳐 계산 완료 ----
----000877 종목 입력피쳐 계산 완료 ----
----000895 종목 입력피쳐 계산 완료 ----
----000938 종목 입력피쳐 계산 완료 ----
----000963 종목 입력피쳐 계산 완료 ----
----000977

----603799 종목 입력피쳐 계산 완료 ----
----603806 종목 입력피쳐 계산 완료 ----
----603833 종목 입력피쳐 계산 완료 ----
----603882 종목 입력피쳐 계산 완료 ----
----603899 종목 입력피쳐 계산 완료 ----
----603986 종목 입력피쳐 계산 완료 ----
----603993 종목 입력피쳐 계산 완료 ----
----605117 종목 입력피쳐 계산 완료 ----
----605499 종목 입력피쳐 계산 완료 ----
----688005 종목 입력피쳐 계산 완료 ----
----688008 종목 입력피쳐 계산 완료 ----
----688012 종목 입력피쳐 계산 완료 ----
----688036 종목 입력피쳐 계산 완료 ----
----688065 종목 입력피쳐 계산 완료 ----
----688111 종목 입력피쳐 계산 완료 ----
----688126 종목 입력피쳐 계산 완료 ----
----688169 종목 입력피쳐 계산 완료 ----
----688187 종목 입력피쳐 계산 완료 ----
----688303 종목 입력피쳐 계산 완료 ----
----688363 종목 입력피쳐 계산 완료 ----
----688396 종목 입력피쳐 계산 완료 ----
----688561 종목 입력피쳐 계산 완료 ----
----688599 종목 입력피쳐 계산 완료 ----
----688981 종목 입력피쳐 계산 완료 ----


In [2]:
AllStock

,TradingDate,Symbol,ShortName,OpenPrice,ClosePrice,HighPrice,LowPrice,Volume,ChangeRatio,MA5,...,Grad120,VMA120,VGrad120,Dis120,Div120,VDis120,VDiv120,CandleType,PriceChange,Target
0,2016-05-02,1,平安银行,7.982,7.982,7.982,7.982,0,0.00000,0.0000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0,0.000000,0
1,2016-05-03,1,平安银行,7.990,8.065,8.096,7.945,48910210,0.01040,0.0000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1,0.000000,0
2,2016-05-04,1,平安银行,8.065,8.096,8.118,8.050,41243559,0.00384,0.0000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1,0.000000,0
3,2016-05-05,1,平安银行,8.073,8.096,8.096,8.043,23951683,0.00000,0.0000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1,0.000000,0
4,2016-05-06,1,平安银行,8.103,7.945,8.103,7.945,34654514,-0.01865,8.0368,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,2023-04-24,688981,中芯国际,62.500,63.050,64.500,60.110,84698990,0.00398,63.0880,...,0.171583,3.432399e+07,551462.225000,138.862633,17.645417,246.763275,5.037500e+07,1,0.079365,0
723,2023-04-25,688981,中芯国际,63.060,61.370,64.570,60.850,87228753,-0.02665,63.1820,...,0.154250,3.488581e+07,675740.312500,134.667993,15.798667,250.040778,5.234294e+07,0,0.771757,0
724,2023-04-26,688981,中芯国际,60.700,58.160,61.060,56.700,106628690,-0.05231,62.3720,...,0.145833,3.567547e+07,557296.254167,127.228346,12.446917,298.885208,7.095322e+07,0,-6.510207,0
725,2023-04-27,688981,中芯国际,57.300,58.290,59.450,57.010,59302154,0.00224,60.7340,...,0.161917,3.600040e+07,351197.379167,127.095916,12.427000,164.726359,2.330175e+07,1,-12.319495,0


In [3]:
#전체 데이터 중 학습/테스트/검증 3분류를 해서 딥러닝 용으로
#날짜 과거-현재 순으로 정렬
AllStock = AllStock.sort_values(by='TradingDate', ascending=True)  # ascending=True로 설정하면 오름차순으로 정렬

total_rows = len(AllStock)
split1_end = int(0.7 * total_rows)
split2_end = int(0.9 * total_rows)

data_split1 = AllStock[:split1_end]  # 70%의 데이터 = 학습용
data_split2 = AllStock[split1_end:split2_end]  # 20%의 데이터 = 테스트용
data_split3 = AllStock[split2_end:]  # 10%의 데이터 = 검증용

# 데이터를 CSV 파일로 저장
train_filename = engine_name + '_train.csv'
test_filename = engine_name+ '_test.csv'
vali_filename = engine_name + '_vali.csv'

#data_split1.to_csv(os.path.join(directory_path, train_filename), index=False, encoding='utf-8-sig')
#data_split2.to_csv(os.path.join(directory_path, vali_filename), index=False, encoding='utf-8-sig')
#data_split3.to_csv(os.path.join(directory_path, test_filename), index=False, encoding='utf-8-sig')

# 데이터를 TXT 파일로 저장
train_filename_txt = engine_name + '_train.txt'
vali_filename_txt = engine_name+ '_test.txt'
test_filename_txt = engine_name + '_vali.txt'

#data_split1.to_csv(os.path.join(directory_path, train_filename_txt), sep=' ', index=False, header=False, encoding='utf-8-sig')
#data_split2.to_csv(os.path.join(directory_path, vali_filename_txt), sep=' ', index=False, header=False, encoding='utf-8-sig')
#data_split3.to_csv(os.path.join(directory_path, test_filename_txt), sep=' ', index=False, header=False, encoding='utf-8-sig')

In [4]:
from tensorflow.keras import backend as K
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : 
    # round : 
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # target set 0(Negative) or 1(Positive)
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # Prediction result set 0(Negative) or 1(Positive)

    # True Positive
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) 
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : 
    # round :
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) 
    y_target_yn = K.round(K.clip(y_target, 0, 1)) 

    # True Positive
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) 
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

In [5]:
batch_size = 512 
epochs = 1000
input_size  = 48
target_num = 1

In [6]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math
from tensorflow.keras.layers import Dropout, Dense, Activation
from tensorflow.keras.models import load_model 
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

In [7]:
#학습코드 여기다가 만들어보자
start_column_index = 3  # 입력피쳐 시작 열의 인덱스 
end_column_index = -1  # 타겟 전까지의 인덱스 (타겟은 맨 뒤에 있음)
date_column_index = 0

#_x는 input feature만 구성되어 있음
#_y는 target으로만 구성되어 있음
train_x = data_split1.iloc[:, start_column_index:end_column_index]  
train_y = data_split1.iloc[:, end_column_index] 
train_x = train_x.reset_index(drop=True) #index 초기화
train_y = train_y.reset_index(drop=True)

vali_code_arr = data_split2['Symbol'].apply(lambda x: str(x).zfill(6)).tolist()
vali_date_arr = data_split2['TradingDate'].tolist() #예측 결과 날짜랑 코드 뽑으려고 list에 일단 차곡
vali_x = data_split2.iloc[:, start_column_index:end_column_index] 
vali_y = data_split2.iloc[:, end_column_index] 

test_code_arr = data_split3['Symbol'].apply(lambda x: str(x).zfill(6)).tolist()
test_date_arr = data_split3['TradingDate'].tolist()
test_x = data_split3.iloc[:, start_column_index:end_column_index] 
test_y = data_split3.iloc[:, end_column_index] 

In [ ]:
with tf.device('/GPU:0'):
    train_x = np.array(train_x).reshape(-1, input_size) #2차원 배열로 바꾸기
    test_x = np.array(test_x).reshape(-1, input_size)
    vali_x =  np.array(vali_x).reshape(-1, input_size)

    train_x = tf.convert_to_tensor(train_x, dtype=tf.float32)
    train_y = tf.convert_to_tensor(train_y, dtype=tf.float32)

    vali_x = tf.convert_to_tensor(vali_x, dtype=tf.float32)
    vali_y = tf.convert_to_tensor(vali_y, dtype=tf.float32)

    test_x = tf.convert_to_tensor(test_x, dtype=tf.float32)
    test_y = tf.convert_to_tensor(test_y, dtype=tf.float32)

    #train_x_conv = np.array(train_x).reshape(train_x.shape[0], train_x.shape[1], 1) #컨볼루션 용으로 바꾸기
    #vali_x_conv = np.array(vali_x).reshape(vali_x.shape[0], vali_x.shape[1], 1)

In [ ]:
print(len(train_x))
print(len(vali_x))
print(len(test_x))

In [ ]:
with tf.device('/GPU:0'):

    model = Sequential()
    
    # 1D 컨볼루션 레이어 추가
    #model.add(Conv1D(64, kernel_size=3, activation='relu',input_shape=(train_x.shape[1], 1)))
    # MaxPooling 레이어 (선택적)
    #model.add(MaxPooling1D(pool_size=2))
    # 데이터 평탄화
    #model.add(Flatten())
    # 완전 연결 레이어 추가
    #model.add(Dense(64, activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    
    model.add(Dense(input_size, activation='sigmoid', input_shape=(input_size,))) 
    model.add(Dense(50,  activation='sigmoid')) 
    model.add(Dropout(0.5))
    model.add(Dense(200,  activation='sigmoid')) 
    model.add(Dropout(0.5))
    model.add(Dense(target_num, activation='sigmoid'))
    
    #model.add(Dense(64, activation='relu', input_shape=(input_size,))) 
    #model.add(Dropout(0.5))
    #model.add(Dense(32, activation='relu'))
    #model.add(Dropout(0.5))
    #model.add(Dense(1, activation='sigmoid'))  # 이진 분류에 맞는 출력 레이어
    
    model.summary()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', precision, recall, f1score])
    
    # Fit and evaluate model 
    history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=1, 
                        validation_data = (vali_x, vali_y))

In [ ]:
## summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()